In [6]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

# You can also import other modules you might need for data preprocessing and evaluation.


In [7]:
model = InceptionV3(weights='imagenet')

# Function to preprocess an image
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(299, 299))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

96112376/96112376 [==============================] - 206s 2us/step


In [17]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the InceptionV3 model without the top classification layer
base_model = InceptionV3(weights='imagenet', include_top=False)

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Create the custom model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the layers of the base model (optional)
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Define data generators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    'C:/Users/ponra/Documents/fire-detection/dataset/train',
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    'C:/Users/ponra/Documents/fire-detection/dataset/test',
    target_size=(299, 299),
    batch_size=32,
    class_mode='binary'
)

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,  # Adjust the number of epochs as needed
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

# Save the trained model
model.save('fire_detection_model.h5')


Found 3252 images belonging to 2 classes.
Found 239 images belonging to 2 classes.
Epoch 1/10
102/102 [==============================] - 311s 3s/step - loss: 0.4421 - accuracy: 0.8561 - val_loss: 0.1911 - val_accuracy: 0.9582
Epoch 2/10
102/102 [==============================] - 274s 3s/step - loss: 0.2166 - accuracy: 0.9207 - val_loss: 0.3710 - val_accuracy: 0.8787
Epoch 3/10
102/102 [==============================] - 473s 5s/step - loss: 0.2091 - accuracy: 0.9253 - val_loss: 0.4479 - val_accuracy: 0.8577
Epoch 4/10
102/102 [==============================] - 325s 3s/step - loss: 0.1968 - accuracy: 0.9284 - val_loss: 0.3299 - val_accuracy: 0.9205
Epoch 5/10
102/102 [==============================] - 295s 3s/step - loss: 0.1984 - accuracy: 0.9262 - val_loss: 0.2831 - val_accuracy: 0.9331
Epoch 6/10
102/102 [==============================] - 1086s 11s/step - loss: 0.1882 - accuracy: 0.9299 - val_loss: 0.3275 - val_accuracy: 0.9372
Epoch 7/10
102/102 [==============================] - 325

C:\Users\ponra\anaconda3\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [37]:
test_folder='C:/Users/ponra/Documents/fire-detection/dataset/test/fire'
test_image_files = [os.path.join(test_folder, filename) for filename in os.listdir(test_folder)]

# Make predictions for each test image
for image_path in test_image_files:
    # Preprocess the image
    input_image = preprocess_image(image_path)
    
    # Make a prediction
    predictions = model.predict(input_image)
    
    # Interpret the predictions (e.g., for binary classification)
    print(predictions[0])
    if predictions[0] >= 0.5:
        print(f"{image_path}: No Fire detected.")
    else:
        print(f"{image_path}: fire detected.")


1/1 [==============================] - 0s 164ms/step
[0.3048424]
C:/Users/ponra/Documents/fire-detection/dataset/test/fire\75.jpg: fire detected.
1/1 [==============================] - 0s 130ms/step
[0.02770374]
C:/Users/ponra/Documents/fire-detection/dataset/test/fire\80.jpg: fire detected.
1/1 [==============================] - 0s 159ms/step
[0.2617141]
C:/Users/ponra/Documents/fire-detection/dataset/test/fire\81.jpg: fire detected.
1/1 [==============================] - 0s 187ms/step
[0.08950327]
C:/Users/ponra/Documents/fire-detection/dataset/test/fire\82.jpg: fire detected.
1/1 [==============================] - 0s 125ms/step
[0.11547206]
C:/Users/ponra/Documents/fire-detection/dataset/test/fire\83.jpg: fire detected.
1/1 [==============================] - 0s 125ms/step
[0.15496618]
C:/Users/ponra/Documents/fire-detection/dataset/test/fire\85.jpg: fire detected.
1/1 [==============================] - 0s 122ms/step
[0.17260528]
C:/Users/ponra/Documents/fire-detection/dataset/test/f